Annisa Muzdalifa <br>
23159011

In [1]:
import numpy as np
from rnn_utils import *

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.dot(Wax, xt)+np.dot(Waa, a_prev)+ba)
    yt_pred = softmax(np.dot(Wya, a_next)+by)
    
    cache= (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [3]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5, 10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by":by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4]", a_next[4])
print("a_next.shape", a_next.shape)
print("yt_pred", yt_pred[1])
print("yt_pred.shape", yt_pred.shape)

a_next[4] [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape (5, 10)
yt_pred [0.9888161  0.01682021 0.21140899 0.36817467 0.98988387 0.88945212
 0.36920224 0.9966312  0.9982559  0.17746526]
yt_pred.shape (2, 10)


In [4]:
def rnn_forward(x, a0, parameters):
    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    a_next = a0
    
    for t in range(0, T_x):
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        a[:, :, t] = a_next
        y_pred[:, :, t] = yt_pred
        caches.append(cache)
        
    caches = (caches, x)
    return a, y_pred, caches

In [5]:
np.random.seed(1)
x = np.random.randn(3, 10, 4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by":by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1]", a[4][1])
print("a.shape", a.shape)
print("y_pred[1][3]", y_pred[1][3])
print("y_pred.shape", y_pred.shape)
print("caches[1][1][3]", caches[1][1][3])
print("len(cache)", len(caches))

a[4][1] [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape (5, 10, 4)
y_pred[1][3] [0.79560373 0.86224861 0.11118257 0.81515947]
y_pred.shape (2, 10, 4)
caches[1][1][3] [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(cache) 2


In [6]:
def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    concat = np.zeros((n_x  + n_a, m))
    concat[:n_a, :] = a_prev
    concat[n_a:, :] = xt
    
    ft = sigmoid(np.dot(Wf, concat)+bf)
    it = sigmoid(np.dot(Wi, concat)+bi)
    cct = np.tanh(np.dot(Wc, concat)+bc)
    c_next = c_prev* ft + it*cct
    ot = sigmoid(np.dot(Wo, concat)+bo)
    a_next = ot*np.tanh(c_next)
    
    yt_pred = softmax(np.dot(Wy, a_next)+by)
    
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)
    
    return a_next, c_next, yt_pred, cache

In [7]:
np.random.seed(1)
xt = np.random.randn(3, 10)
a_prev = np.random.randn(5, 10)
c_prev = np.random.randn(5, 10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5, 1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5, 1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5, 1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5, 1)
Wy = np.random.randn(2, 5)
by = np.random.randn(1, 1)

parameters = {"Wf": Wf, "bf": bf, "Wi": Wi, "bi":bi, "Wo":Wo, "bo":bo, "Wc":Wc, "bc":bc, "Wy":Wy, "by":by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)

print("a_next[4]", a_next[4])
print("a_next.shape", a_next.shape)
print("c_next[2]", c_next[2])
print("c_next.shape", c_next.shape)
print("yt[1]", yt[1])
print("yt.shape", yt.shape)
print("cache[1][3]", cache[1][3])
print("len(cache)", len(cache))

a_next[4] [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape (5, 10)
c_next[2] [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape (5, 10)
yt[1] [0.88515863 0.2693483  0.35881369 0.26375541 0.9845904  0.46704976
 0.01810876 0.21934384 0.55723129 0.14129154]
yt.shape (2, 10)
cache[1][3] [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) 10


In [8]:
def lstm_forward(x, a0, parameters):
    caches = []
    
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    for t in range(0, T_x):
        a_next, c_next, yt, cache = lstm_cell_forward(x[:, :, t], a_next, c_next, parameters)
        a[:, :, t] = a_next
        c[:, :, t] = c_next
        y[:, :, t] = yt
        caches.append(cache)
        
    caches = (caches, x)
    
    return a, y, c, caches

In [9]:
np.random.seed(1)
x = np.random.randn(3, 10, 7)
a0 = np.random.randn(5, 10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5, 1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5, 1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5, 1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5, 1)
Wy = np.random.randn(2, 5)
by = np.random.randn(2, 1)

parameters = {"Wf": Wf, "bf": bf, "Wi": Wi, "bi":bi, "Wo":Wo, "bo":bo, "Wc":Wc, "bc":bc, "Wy":Wy, "by":by}

a, y, c, caches = lstm_forward(x, a0, parameters)

print("a[4][3][6]", a[4][3][6])
print("a.shape", a.shape)
print("y[1][4][3]", y[1][4][3])
print("y.shape", y.shape)
print("caches[1][1][1]", caches[1][1][1])
print("c[1][2][1]", c[1][2][1])
print("len(cache)", len(cache))

a[4][3][6] 0.17211776753291672
a.shape (5, 10, 7)
y[1][4][3] 0.9508734618501101
y.shape (2, 10, 7)
caches[1][1][1] [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.8555449167181981
len(cache) 10


In [10]:
def rnn_cell_backward(da_next, cache):
    (a_next, a_prev, xt, parameters) = cache
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    dtanh = 1 - np.power(a_next, 2)
    
    dxt = np.dot(Wax.T, da_next*dtanh)
    dWax = np.dot(da_next*dtanh, xt.T)
    
    da_prev = np.dot(Waa.T, da_next*dtanh)
    dWaa = np.dot(da_next*dtanh, a_prev.T)
    
    dba = np.sum(da_next*dtanh, axis=1, keepdims=True)
    
    gradients = {"dxt":dxt, "dWax": dWax, "da_prev":da_prev, "dWaa": dWaa, "dba":dba}
    
    return gradients

In [11]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5, 10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by":by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)

da_next = np.random.randn(5, 10)
gradients = rnn_cell_backward(da_next, cache)

print("gradients['dxt'][1][2]=", gradients['dxt'][1][2])
print("gradients['dxt'].shape=", gradients['dxt'].shape)
print("gradients['da_prev'][2][3]=", gradients['da_prev'][2][3])
print("gradients['da_prev'].shape=", gradients['da_prev'].shape)
print("gradients['dWax'][3][1]=", gradients['dWax'][3][1])
print("gradients['dWax'].shape=", gradients['dWax'].shape)
print("gradients['dWaa'][1][2]=", gradients['dWaa'][1][2])
print("gradients['dWaa'].shape=", gradients['dWaa'].shape)
print("gradients['dba'][4]=", gradients['dba'][4])
print("gradients['dba'].shape=", gradients['dba'].shape)

gradients['dxt'][1][2]= 1.3653821219712916
gradients['dxt'].shape= (3, 10)
gradients['da_prev'][2][3]= -0.043577791064616234
gradients['da_prev'].shape= (5, 10)
gradients['dWax'][3][1]= -1.5012584841864745
gradients['dWax'].shape= (5, 3)
gradients['dWaa'][1][2]= 1.1441951795389382
gradients['dWaa'].shape= (5, 5)
gradients['dba'][4]= [1.42397243]
gradients['dba'].shape= (5, 1)


In [12]:
def rnn_backward(da, caches):
    (caches, x) = caches
    (a1, a0, x1, parameters) = caches[1]
    
    n_a, m, T_x = da.shape
    n_x, m = x1.shape
    
    dx = np.zeros((n_x, m, T_x))
    dWax = np.zeros((n_a, n_x))
    dba = np.zeros((n_a, 1))
    dWaa = np.zeros((n_a, n_a))
    da0 = np.zeros((n_a, m))
    da_prevt = np.zeros((n_a, m))
    for t in reversed(range(0, T_x)):
        gradients = rnn_cell_backward(da[:, :, t]+da_prevt, caches[t])
        
        dxt, da_prevt, dWaxt, dbat, dWaat = gradients["dxt"], gradients["da_prev"], gradients["dWax"], gradients["dba"], gradients["dWaa"]
        dx[:, :, t] = dxt
        dWax += dWaxt
        dWaa += dWaat
        dba += dbat
        
    da0 = da_prevt
    gradients = {"dx": dx, "dWax": dWax, "dWaa": dWaa, "dba":dba, "da0":da0}
    return gradients

In [16]:
np.random.seed(1)
x = np.random.randn(3,10, 4)
a0 = np.random.randn(5, 10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by":by}

a, y, caches = rnn_forward(x, a0, parameters)
da = np.random.randn(5, 10, 4)
gradients = rnn_backward(da, caches)

print("gradients['dx'][1][2]=", gradients['dx'][1][2])
print("gradients['dx'].shape=", gradients['dx'].shape)
print("gradients['da0'][2][3]=", gradients['da0'][2][3])
print("gradients['da0'].shape=", gradients['da0'].shape)
print("gradients['dWax'][3][1]=", gradients['dWax'][3][1])
print("gradients['dWax'].shape=", gradients['dWax'].shape)
print("gradients['dWaa'][1][2]=", gradients['dWaa'][1][2])
print("gradients['dWaa'].shape=", gradients['dWaa'].shape)
print("gradients['dba'][4]=", gradients['dba'][4])
print("gradients['dba'].shape=", gradients['dba'].shape)

gradients['dx'][1][2]= [-0.86050481 -0.14439617 -0.02986862  0.10659932]
gradients['dx'].shape= (3, 10, 4)
gradients['da0'][2][3]= 0.0057969143465314895
gradients['da0'].shape= (5, 10)
gradients['dWax'][3][1]= 0.44296398799038544
gradients['dWax'].shape= (5, 3)
gradients['dWaa'][1][2]= 0.44183867362064544
gradients['dWaa'].shape= (5, 5)
gradients['dba'][4]= [-3.51296232]
gradients['dba'].shape= (5, 1)


In [19]:
print(parameters['Waa'])
print(gradients['dWaa'])

[[-0.64691669  0.90148689  2.52832571 -0.24863478  0.04366899]
 [-0.22631424  1.33145711 -0.28730786  0.68006984 -0.3198016 ]
 [-1.27255876  0.31354772  0.50318481  1.29322588 -0.11044703]
 [-0.61736206  0.5627611   0.24073709  0.28066508 -0.0731127 ]
 [ 1.16033857  0.36949272  1.90465871  1.1110567   0.6590498 ]]
[[ 0.5790693   1.00989782  0.45607653  0.47247531  0.42070208]
 [-4.0169478   1.27988543  0.44183867 -3.005149    3.68367168]
 [-1.58623708 -1.82523051  1.88662745  1.25372645  1.31696213]
 [-4.06220832 -2.19095447 -2.34204406 -2.90812842  3.40599372]
 [-1.52082597 -3.22930103 -3.15030367  2.82807664  0.79115841]]


In [14]:
def lstm_cell_backward(da_next, dc_next, cache):
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
    
    n_x, m = xt.shape
    n_a, m = a_next.shape
    
    dot = da_next * np.tanh(c_next)
    dcct = (da_next*ot * (1-np.power(np.tanh(c_next), 2)) +dc_next) * it
    dit = (da_next*ot * (1-np.power(np.tanh(c_next), 2))+dc_next) * cct
    dft = (da_next*ot * (1-np.power(np.tanh(c_next), 2))+dc_next) * c_prev
    
    dit = dit*it*(1-it)
    dft = dft*ft*(1-ft)
    dot = dot*ot*(1-ot)
    dcct = 1 * (1-np.power(cct, 2))
    
    concat = np.zeros((n_x + n_a, m))
    concat[:n_a, :] = a_prev
    concat[n_a:, :] = xt
    dWf = np.dot(dft, concat.T)
    dWi = np.dot(dit, concat.T)
    dWc = np.dot(dcct, concat.T)
    dWo = np.dot(dot, concat.T)
    dbf = np.sum(dft, axis=1, keepdims=True)
    dbi = np.sum(dit, axis=1, keepdims=True)
    dbc = np.sum(dcct, axis=1 , keepdims=True)
    dbo = np.sum(dot, axis=1, keepdims=True)
    
    da_prevx = np.dot(parameters["Wf"].T, dft) + np.dot(parameters["Wo"].T, dot) + np.dot(parameters["Wi"].T, dit) + np.dot(parameters["Wc"].T, dcct)
    da_prev = da_prevx[:n_a, :]
    dc_prev = (da_next* ot * (1-np.power(np.tanh(c_next), 2))+dc_next) * ft
    dxt= da_prevx[n_a:, :]
    
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev":dc_prev,
                "dWf": dWf, "dWi": dWi, "dWc": dWc, "dWo": dWo,
                "dbf": dbf, "dbi": dbi, "dbo": dbo, "dbc": dbc}
    
    return gradients

In [20]:
np.random.seed(1)
xt = np.random.randn(3, 10)
a_prev = np.random.randn(5, 10)
c_prev = np.random.randn(5, 10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5, 1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5, 1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5, 1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5, 1)
Wy = np.random.randn(2, 5)
by = np.random.randn(2, 1)

parameters = {"Wf": Wf, "bf": bf, "Wi": Wi, "bi":bi, "Wo":Wo, "bo":bo, "Wc":Wc, "bc":bc, "Wy":Wy, "by":by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)

da_next = np.random.randn(5, 10)
dc_next = np.random.randn(5, 10)
gradients = lstm_cell_backward(da_next, dc_next, cache)

print("gradients['dxt'][1][2]=", gradients['dxt'][1][2])
print("gradients['dxt'].shape=", gradients['dxt'].shape)
print("gradients['da_prev'][2][3]=", gradients['da_prev'][2][3])
print("gradients['da_prev'].shape=", gradients['da_prev'].shape)
print("gradients['dc_prev'][2][3]=", gradients['dc_prev'][2][3])
print("gradients['dc_prev'].shape=", gradients['dc_prev'].shape)
print("gradients['dWf'][3][1]=", gradients['dWf'][3][1])
print("gradients['dWf'].shape=", gradients['dWf'].shape)
print("gradients['dWi'][1][2]=", gradients['dWi'][1][2])
print("gradients['dWi'].shape=", gradients['dWi'].shape)
print("gradients['dWc'][3][1]=", gradients['dWc'][3][1])
print("gradients['dWc'].shape=", gradients['dWc'].shape)
print("gradients['dWo'][1][2]=", gradients['dWo'][1][2])
print("gradients['dWo'].shape=", gradients['dWo'].shape)
print("gradients['dbf'][4]=", gradients['dbf'][4])
print("gradients['dbf'].shape=", gradients['dbf'].shape)
print("gradients['dbi'][4]=", gradients['dbi'][4])
print("gradients['dbi'].shape=", gradients['dbi'].shape)
print("gradients['dbc'][4]=", gradients['dbc'][4])
print("gradients['dbc'].shape=", gradients['dbc'].shape)
print("gradients['dbo'][4]=", gradients['dbo'][4])
print("gradients['dbo'].shape=", gradients['dbo'].shape)

gradients['dxt'][1][2]= 1.776382835953809
gradients['dxt'].shape= (3, 10)
gradients['da_prev'][2][3]= -0.19660751310831898
gradients['da_prev'].shape= (5, 10)
gradients['dc_prev'][2][3]= 0.7975220387970015
gradients['dc_prev'].shape= (5, 10)
gradients['dWf'][3][1]= -0.1479548381644968
gradients['dWf'].shape= (5, 8)
gradients['dWi'][1][2]= 1.0574980552259903
gradients['dWi'].shape= (5, 8)
gradients['dWc'][3][1]= 3.7730841791634275
gradients['dWc'].shape= (5, 8)
gradients['dWo'][1][2]= 0.3313115952892109
gradients['dWo'].shape= (5, 8)
gradients['dbf'][4]= [0.18864637]
gradients['dbf'].shape= (5, 1)
gradients['dbi'][4]= [-0.40142491]
gradients['dbi'].shape= (5, 1)
gradients['dbc'][4]= [2.00933438]
gradients['dbc'].shape= (5, 1)
gradients['dbo'][4]= [0.13893342]
gradients['dbo'].shape= (5, 1)


In [21]:
print(Wf)
print(gradients['dWf'])

[[-0.1809203  -0.60392063 -1.23005814  0.5505375   0.79280687 -0.62353073
   0.52057634 -1.14434139]
 [ 0.80186103  0.0465673  -0.18656977 -0.10174587  0.86888616  0.75041164
   0.52946532  0.13770121]
 [ 0.07782113  0.61838026  0.23249456  0.68255141 -0.31011677 -2.43483776
   1.0388246   2.18697965]
 [ 0.44136444 -0.10015523 -0.13644474 -0.11905419  0.01740941 -1.12201873
  -0.51709446 -0.99702683]
 [ 0.24879916 -0.29664115  0.49521132 -0.17470316  0.98633519  0.2135339
   2.19069973 -1.89636092]]
[[-0.1421699   0.37848152  0.11926438  0.47170657  0.28198475  0.03379451
  -0.14190739  0.11263537]
 [ 0.33483087  0.28531747  0.19488761  0.49997521  0.71242185 -0.43398358
  -0.65500412  0.44600686]
 [-0.06376814 -0.06822228 -0.00809406  0.01610937 -0.05571127  0.14833691
   0.10106896  0.04564574]
 [ 0.0621335  -0.14795484  0.12115521 -0.7750645   0.0157104  -0.20879551
  -0.02786287  0.39423542]
 [ 0.05048145 -0.49088288  0.4095815   0.03122771 -0.33133813  0.88204865
   0.55665687  0.